In [2]:
import polars as pl
print(pl.__version__)

import os

1.18.0


## Get list of files

In [3]:
parquet_folder = "/home/vikas/Desktop/Globus/Gaia/gaia_parquet"

def get_files(table_name):

    return os.path.join(parquet_folder, table_name, "*")

## Run query

#### Eager

In [4]:
# df_photometry = (
#     pl.read_parquet(get_files("photometry"))
#     .filter(pl.col("phot_bp_mean_flux") > 5)
# )

#### Lazy

##### Number of unique objects

In [5]:
def stream_query_1():

    q1 = (
        pl.scan_parquet(get_files("photometry"))
          .select("object_id")
          .unique()
    )

    #print(q1.explain(streaming = True))

    return q1.collect(streaming = True)

In [6]:
#stream_query_1().shape

##### Maximum and 

In [9]:
def stream_query_2():

    q1 = (
        pl.scan_parquet(get_files("photometry"))
          .group_by("healpix")
          .agg(
              pl.col("phot_g_mean_mag").min().alias("min_phot_g_mean_mag"),
              pl.col("phot_g_mean_mag").max().alias("max_phot_g_mean_mag")
          )
        .sort("healpix")
    )

    #print(q1.explain(streaming = True))

    return q1.collect(streaming = True)

In [10]:
stream_query_2()

healpix,min_phot_g_mean_mag,max_phot_g_mean_mag
i32,f32,f32
0,6.855546,19.842781
1,7.355467,19.706682
2,7.312822,19.992142
3,6.964732,19.499153
4,3.382374,20.126875
…,…,…
2685,6.030447,20.092436
2686,4.124845,20.017384
2687,4.425611,20.595608
